# Import code from gitHub

In [0]:
import json
import subprocess
import sys
import time

import numpy as np
from IPython.core.display import display, HTML
from google.colab import files

!pip install overrides

Ничто = None
_git_branch = 'dev'


def exec(x):
  r = subprocess.check_output(x, shell=True)
  r = r.decode('unicode-escape').encode('latin1').decode('utf8')
  print(r)


print(f"fetching code from GitHub.....{_git_branch}")
try:
  exec('rm -r nlp_tools')
except:
  pass
exec(f'git clone --single-branch --branch {_git_branch} https://github.com/nemoware/analyser.git nlp_tools')

print('🦊 GIT revision:')
exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

sys.path.insert(0, 'nlp_tools')

print('❤️importing Code from GitHub ... DONE')


#----
import matplotlib as mpl
from documents import TextMap
from renderer import HtmlRenderer
from legal_docs import DocumentJson
 

class DemoRenderer(HtmlRenderer):
  def render_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
    html = self.to_color_text(tokens, weights, colormap, print_debug, _range)
    display(HTML(html))

  def to_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
    return super()._to_color_text(tokens, weights, mpl, colormap=colormap, _range=_range)

   
renderer_ = DemoRenderer()

def print_json_summary(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['$words'])
  print(f'read file {cd.filename}')

  for tag in cd.tags:
    span = tag.span
    _map = cd.tokenization_maps[tag.span_map]
    print(tag)
 
  

### imports

In [0]:
import pickle
import unittest
import warnings

!pip install pyjarowinkler

from contract_parser import ContractAnlysingContext, ContractDocument
from contract_patterns import ContractPatternFactory
from documents import TextMap
from legal_docs import LegalDocument
from legal_docs import DocumentJson

from ml_tools import SemanticTag, filter_values_by_key_prefix

import nltk
nltk.download('punkt')


 

### Utilits

In [0]:
def print_semantic_tag(tag: SemanticTag, map: TextMap):
  print('-->', f'{tag.kind} \t\t [{tag.value}]\t [{map.text_range(tag.span)}]')


def print_json_summary(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['$words'])
 
  for tag in cd.tags:
    if tag.kind !='headline':
     print_semantic_tag(tag, wordsmap)
 


def json2html(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['words'])
  markup_vector = np.zeros(len(wordsmap))
 
  for tag in cd.attributes:
    if 'span' in  cd.attributes[tag]:
      span = cd.attributes[tag]['span']
      confidence = cd.attributes[tag]['confidence']
      markup_vector[span[0]:span[1]] += confidence
    else:
      warnings.warn(f'{tag} has no span')

  return renderer_.to_color_text(wordsmap.tokens, markup_vector, _range=(0, 1))
  
 
def load_and_show_json(filename):
  with open(filename, 'r') as json_file:
    data_str = json_file.read()
    # print(str(data_str))
    json_obj:DocumentJson = DocumentJson.from_json( data_str)
    html = json2html(json_obj)
    display(HTML(html))




## Export all contracts to JSON

### Mount gDrive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [0]:
from embedding_tools import ElmoEmbedder
elmo_embedder = ElmoEmbedder(module_url = 'https://storage.googleapis.com/az-nlp/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz')
import tensorflow as tf
import tensorflow_hub as hub
import nltk
nltk.download('punkt')
from contract_parser import ContractAnlysingContext
contractAnlysingContext = ContractAnlysingContext(elmo_embedder, renderer_)



### Init document-parser lib

In [0]:
lib_version='1.1.3'

!wget https://github.com/nemoware/document-parser/releases/download/$lib_version/document-parser-$lib_version-distribution.zip
!unzip document-parser-$lib_version-distribution.zip

import os

os.environ ['documentparser']=f'/content/document-parser-{lib_version}'
from integration.word_document_parser import WordDocParser, join_paragraphs
wp = WordDocParser()

##### Parse all

In [0]:
attempt = 2

In [0]:
import datetime
attempt += 1
dirname = '{0:%y.%m.%d}'.format(datetime.datetime.now())
_out = f'/content/gdrive/My Drive/GazpromOil/Всяческая документация (shared)/JSONs/contracts-{dirname}-{attempt}/'
!mkdir '$_out'
print(_out)

In [0]:
from legal_docs import DocumentJson

contracts_filename_prefix = '/content/gdrive/My Drive/GazpromOil/Contracts/'

filenames = wp.list_filenames(contracts_filename_prefix)
filenames = ['/content/gdrive/My Drive/GazpromOil/Contracts/3 этап по правилам/! Договор _1.docx']
print(filenames)

cnt = 0
for fn in filenames:
  cnt += 1
  print(f'reading:"{fn}"')
  short_fn = fn.split('/')[-1]

  try:
    # ------------------------
    results = wp.read_doc(fn)
    k=0
    for doc in results['documents'][:1]:
      k+=1
      contract = join_paragraphs(doc, fn)
      # ------------------------

      contractAnlysingContext.analyze_contract_doc(contract)
      contract = contractAnlysingContext.contract
      
      _path =  os.path.join(_out, f'{k}-{short_fn}.json')
      with open(_path, 'w') as file:
        # 
        jjj = DocumentJson(contract)
        file.write(jjj.dumps())
        print(f'saved file to {_path}')
  except Exception as e:
    print(e)


### Verify random json files

In [0]:
# test_json_file = os.path.join(_out,filenames[3]).split('/')[-1]+'.json'
test_json_file = '/content/gdrive/My Drive/GazpromOil/Всяческая документация (shared)/JSONs/contracts-19.10.17-4/1-! Договор _1.docx.json'
load_and_show_json(test_json_file)
 

#Parse 2K sectioned contracts (Achtung, dis takes time)
- upload zipped results of document-parser first 

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
!unzip /content/jsons_phase0.zip

In [0]:
import glob
filenames = [file for file in glob.glob( "/content/jsons_phase0/**/*.json", recursive=True)] 


In [0]:
!mkdir out

In [0]:

import csv, json
from bson import json_util


from contract_parser import ContractAnlysingContext
from integration.word_document_parser import join_paragraphs


def export_csv(rows, headline=['1', '2', '3', '4', '5', '6', '7', '8', '9']):
  with open(f'contracts-stats.csv', mode='w') as csv_file:
    _writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    _writer.writerow(headline)
    for l in rows:
      _writer.writerow(l)


def _parse_contract(res, doc_id, row, ctx: ContractAnlysingContext):
  contract = join_paragraphs(res, doc_id)
  ctx.analyze_contract_doc(contract)

  row[4:8] = [contract.tag_value('org.1.name'),
              contract.tag_value('org.1.alias'),
              contract.tag_value('org.2.name'),
              contract.tag_value('org.2.alias')]
  return contract

 
rows=[]
cnt=0

for fn in filenames[200:]:
  with open(fn, 'r') as file:
    json_string = file.read()
    res = json.loads(json_string, object_hook=json_util.object_hook)

  if 'CONTRACT' == res["documentType"]:
    row = [cnt, '', None, None, None, None, None, None, fn, None]
    row[2:4] = [res["documentType"], res["documentDate"]]
    contract = _parse_contract(res, fn,  row, contractAnlysingContext)
         
    short_fn = fn.split('/')[-1]
    with open(f'/content/out/{short_fn}.json', 'w') as file:
      jjj = DocumentJson(contract)
      file.write(jjj.dumps())


    rows.append(row)
    if cnt % 5 == 0: #save every 5 rows
      export_csv(rows)

  cnt+=1

In [0]:
cnt

In [0]:
import zipfile
import os
import sys

def zipfolder(foldername, target_dir):            
  zipobj = zipfile.ZipFile(foldername + '.zip', 'w', zipfile.ZIP_DEFLATED)
  rootlen = len(target_dir) + 1
  for base, dirs, files in os.walk(target_dir):
    for file in files:
      fn = os.path.join(base, file)
      zipobj.write(fn, fn[rootlen:])



zipfolder('out','/content/out')